<a href="https://colab.research.google.com/github/ehersch/MNIST_Demo/blob/main/mnist_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.08MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 134kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.27MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.65MB/s]


In [9]:
# Old image w/o using a transform (just loads in raw image)
# trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=none)
trainset[0][0]

In [16]:
class Model(nn.Module):
    def __init__(self, hidden):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(64 * 5 * 5, hidden)

        self.pool = nn.MaxPool2d(2,2)
        self.fc2 = nn.Linear(hidden, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [17]:
cnn_model = Model(100)
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001)

In [18]:
if torch.cuda.is_available():
    cnn_model.cuda()

In [19]:
def train(model, train_loader, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}\n-------------------------------")

        for batch, (X, y) in enumerate(train_loader):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(X)
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            if batch % 100 == 0:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{len(train_loader.dataset):>5d}]")

In [20]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [21]:
train(cnn_model, train_loader=train_loader, optimizer=optimizer, epochs=10)

Epoch 1
-------------------------------
loss: 2.309242  [    0/60000]
loss: 0.364770  [ 6400/60000]
loss: 0.161665  [12800/60000]
loss: 0.180593  [19200/60000]
loss: 0.050899  [25600/60000]
loss: 0.130594  [32000/60000]
loss: 0.051690  [38400/60000]
loss: 0.103741  [44800/60000]
loss: 0.169199  [51200/60000]
loss: 0.030066  [57600/60000]
Epoch 2
-------------------------------
loss: 0.067859  [    0/60000]
loss: 0.074695  [ 6400/60000]
loss: 0.012466  [12800/60000]
loss: 0.077722  [19200/60000]
loss: 0.023760  [25600/60000]
loss: 0.180173  [32000/60000]
loss: 0.126627  [38400/60000]
loss: 0.059434  [44800/60000]
loss: 0.004706  [51200/60000]
loss: 0.011970  [57600/60000]
Epoch 3
-------------------------------
loss: 0.022486  [    0/60000]
loss: 0.005330  [ 6400/60000]
loss: 0.016531  [12800/60000]
loss: 0.016631  [19200/60000]
loss: 0.134596  [25600/60000]
loss: 0.049288  [32000/60000]
loss: 0.020986  [38400/60000]
loss: 0.015411  [44800/60000]
loss: 0.033514  [51200/60000]
loss: 0.00

In [22]:
test_loader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True)

In [23]:
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)

            outputs = model(X)                 # (batch_size, num_classes)
            preds = outputs.argmax(dim=1)      # predicted class indices

            correct += (preds == y).sum().item()
            total += y.size(0)

    print(f"Accuracy: {correct / total:.4f}")


In [24]:
test(cnn_model, test_loader)

Accuracy: 0.9978
